# Stress gradient calculation

This demo shows the stress gradient calculation module. A gradient is calculated by fitting a plane into a node and its neighbor nodes of an FEM mesh.

See [documentation](https://pylife.readthedocs.io/en/latest/mesh/gradient.html) for details on the module.

In [ ]:
import numpy as np
import pandas as pd

import pylife.stress.equistress
import pylife.mesh.gradient
import pylife.vmap

import pyvista as pv

Read in demo data and add the stress tensor dimensions for the third dimension.

In [ ]:
vm_mesh = pylife.vmap.VMAPImport("plate_with_hole.vmap")
mesh = (vm_mesh.make_mesh('1', 'STATE-2')
               .join_coordinates()
               .join_variable('STRESS_CAUCHY')
               .to_frame())

Calculate and add von Mises stress

In [ ]:
mesh['mises'] = mesh.equistress.mises()

Calculate stress gradient on von Mises stress

In [ ]:
grad = mesh.gradient.gradient_of('mises')
grad["abs_grad"] = np.linalg.norm(grad, axis = 1)
mesh = mesh.join(grad)
display(mesh)


In [ ]:
grid = pv.UnstructuredGrid(*mesh.mesh.vtk_data())
plotter = pv.Plotter()
plotter.add_mesh(grid, scalars=mesh.groupby('element_id')["abs_grad"].mean().to_numpy(), show_edges=True, cmap='jet')
plotter.add_scalar_bar()
plotter.show()

In [ ]:
"That's it"